In [1]:
## compute entropy of spectrum signals as biomarkers
## udpates on Apr 11 2022:
## save spectrum as time points by frequency for each subject.
## calculate spectrum only on test set.

import glob
import pandas as pd
import numpy as np
import re
import os

## read harmonics and time signals:

## hcp data:
# input_harmonics_wavelets = 'hcp_out02_harmonics_100'
# input_time_signal = '/home/xin/Downloads/FullData_Oct26/Scan1/'
# output_dir = 'hcp_out03_spectrum_power'

## adni data:
input_harmonics_wavelets = 'adni_out02_harmonics'
input_time_signal = '/home/xin/Downloads/BrainImaging_UNC/out04_adni_roi_signals1'
output_dir = 'adni_out03_spectrum_power'

## Notes:
# got NaN values in wavelets (e.g for ROI 3 all values are nan), 
# but time signals does not have nans...
# need to check where those NaNs are derived...

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

harmonic_wavelets_files = glob.glob(input_harmonics_wavelets + '/CommonHarWavelets_Harmonics*.csv')
harmonic_wavelets_files.sort()

## ADNI:
time_signal_files = pd.read_csv('/home/xin/Downloads/Harmonics/adni_out01_time_signal_test_train_split/test_set_roi_signals.csv', header = None)    
time_signal_files = time_signal_files.iloc[:,0].values.tolist()
## HCP data:
# time_signal_files = glob.glob(input_time_signal + '/TimeSeries*.csv')


In [2]:
# for each wavelets (corresponding to a brain region) calcluate inner product with roi signals for each subject:

spectrum_list = [] # nested array with row: brain region and column: subject.
brain_region_list = []
subject_id_list = []

fill_subject_id = True 
test_break = False

for file_wavelets in harmonic_wavelets_files:
    print(file_wavelets)
    spectrum_sub_list = []
    wavelets = pd.read_csv(file_wavelets, header = None)
    
    if wavelets.isna().any(axis = None):
        print('Nan values for wavelets:' + file_wavelets)
        spectrum_list.append(spectrum_sub_list)
        continue
    
    brain_region = file_wavelets[-7:-4]
    brain_region_list.append(brain_region)
    
    for file_time_signal in time_signal_files:
        
        ## adni:
        subject_id = re.search('roi_signals_power264_(.*).txt', file_time_signal).group(1)
        time_signal = pd.read_csv(file_time_signal, header = 0, sep = '\t')
        time_signal.drop(columns = ['Unnamed: 264'], inplace = True)
        time_signal.columns = time_signal.columns.astype(int)
        time_signal = time_signal.sort_index(ascending=True, axis=1)
        
        ## global signal regression:
        time_signal = time_signal.sub(time_signal.mean(axis = 1), axis = 0)
        
        ## hcp:
        # subject_id = re.search('TimeSeries(.*).csv', file_time_signal).group(1)
        # time_signal = pd.read_csv(file_time_signal, header = None, sep = ',')
        # time_signal = time_signal.T
        
        # print(subject_id)

        # check nan values:
        if time_signal.isna().any(axis = None):
            print('Nan value for time_signal:' + subject_id)
            spectrum_sub_list.append([])
            # test_break = True
            continue

        # we could use this to directly skip files with NaN values. not implemented yet...
        if fill_subject_id:
            subject_id_list.append(subject_id)
        
        ## compute inner product for each time point:
        P = np.matmul(time_signal.values, wavelets.values)
        spectrum_sub_list.append(P)
    
#     if test_break:
#         break
        
    fill_subject_id = False
    spectrum_list.append(spectrum_sub_list)
        

adni_out02_harmonics/CommonHarWavelets_Harmonics001.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics002.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics003.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics004.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics005.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics006.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics007.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics008.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics009.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics010.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics011.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics012.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics013.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics014.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics015.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics016.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics017.csv
adni_out02_harmonics/CommonHarWavelets_Harmonics

In [5]:
## save spectrum for each subject and brain region:
        
for brain_index in range(len(spectrum_list)):
    if len(spectrum_list[brain_index])==0:
        continue
    for subj in range(len(spectrum_list[0])):
        P = spectrum_list[brain_index][subj]

        out_file_name = output_dir + '/spectrum_subj' + '{:03}'.format(subj) + '_roi_' + '{:03}'.format(brain_index) + '.csv'
        np.savetxt(out_file_name, np.array(P), delimiter = ',')


In [6]:
len(spectrum_list)

264

In [7]:
## make histogram

## calculate entropy